# Finite Distributed Lag Models

In [3]:
import numpy as np 
import sqlalchemy
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import statsmodels.api as sm

In [2]:
def download_data_from_mongoatlas(database, table, query={}):
    username = 'entropy_scholar'
    password = 'OCN4kEzbsW5lqQ2W'
    url = 'mongodb+srv://{}:{}@entropy-sfneo.mongodb.net/test?retryWrites=true&w=majority'.format(username, password)
    client = MongoClient(url)    
    db = client[database]
    collection = db[table]
    df = pd.DataFrame(list(collection.find(query))).drop('_id', axis=1)
    try:
        df = df.sort_values(['date','clave_ent'], ascending=True)
    except:
        df = df.sort_values(['date'], ascending=True)
    df = df.reset_index(drop=True)
    return df

In [4]:
def growth_lag(data_original, variable, lag):
    data = data_original.copy()
    data[variable] = data[variable].astype(float)
    data['lag'] = data.shift(lag)[variable]
    data[variable+'_growth_lag_'+str(lag)] = (data[variable]/data['lag'] - 1)*100
    data = data.drop('lag', axis=1)
    return data

In [24]:
columna_pib = 'pib'
pib = download_data_from_mongoatlas('inegi_api', 'pib_desestacionalizado_trimestral_a_precios_2013_agregado_nacional')
pib = pib[['date', columna_pib]]
pib = growth_lag(pib, columna_pib, 1)

In [25]:
pib_lagged_1 = []
for i in range(len(pib)):
    try:
        pib_1 = pib['{}_growth_lag_1'.format(columna_pib)][i - 1]
    except:
        pib_1 = np.nan
    pib_lagged_1.append(pib_1)
pib['{}_lagged_1'.format(columna_pib)] = pib_lagged_1
pib = pib.dropna(axis=0, how='any')

In [26]:
pib = pib.loc[(pib['date'] >= '1962/01') & (pib['date'] <= '2012/04')]
X = pib['{}_lagged_1'.format(columna_pib)]
Y = pib['{}_growth_lag_1'.format(columna_pib)]
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:       pib_growth_lag_1   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     15.70
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           0.000166
Time:                        21:13:28   Log-Likelihood:                -132.99
No. Observations:                  78   AIC:                             270.0
Df Residuals:                      76   BIC:                             274.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.3669      0.166      2.216   